In [1]:
# Imports

from itertools import permutations, combinations
import os
import pickle
import platform
import random
from tkinter import Tk

from cvxopt import solvers, matrix
import math
from matplotlib import animation
from  matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import torch
import torchvision
from torchvision import transforms, models,datasets
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# OS related settings
if platform.system() == 'Windows':
    print('Windows')
#     %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    print('macOS')
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    print('Linux')
# This line of "print" must exist right after %matplotlib command, otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')

macOS
Interactive plot activated


In [2]:
# ChIQP

import numpy as np
import itertools
from cvxopt import solvers, matrix

# Silencing solvers.qp
from contextlib import redirect_stdout
from io import StringIO
class NullIO(StringIO):
    def write(self, txt):
        pass


def silent(fn):
    """Decorator to silence functions."""
    def silent_fn(*args, **kwargs):
        with redirect_stdout(NullIO()):
            return fn(*args, **kwargs)
    return silent_fn


class Choquet_Integral_QP:

    def __init__(self):
        """Instantiation of a ChoquetIntegral.
           This sets up the ChI. It doesn't take any input parameters
           because you may want to use pass your own values in(as opposed
           to learning from data). To instatiate, use
           chi = ChoquetIntegral.ChoquetIntegral()
        """
        self.trainSamples, self.trainLabels = [], []
        self.testSamples, self.testLabels = [], []
        self.N, self.numberConstraints, self.M = 0, 0, 0
        self.g = 0
        self.fm = []
        self.type = []


    def train_chi(self, x1, l1):
        """
        This trains this instance of your ChoquetIntegral w.r.t x1 and l1.
        :param x1: These are the training samples of size N x M(inputs x number of samples)
        :param l1: These are the training labels of size 1 x M(label per sample)
        """
        self.type = 'quad'
        self.trainSamples = x1
        self.trainLabels = l1
        self.N = self.trainSamples.shape[0]
        self.M = self.trainSamples.shape[1]
#         print("Number Inputs : ", self.N, "; Number Samples : ", self.M)
        self.fm = self.produce_lattice()

        return self



    def chi_quad(self, x2):
        """
        This will produce an output for this instance of the ChI
        This will use the learned(or specified) Choquet integral to
        produce an output w.r.t. to the new input.
        :param x2: testing sample
        :return: output of the choquet integral.
        """
        if self.type == 'quad':
            n = len(x2)
            pi_i = np.argsort(x2)[::-1][:n] + 1
            ch = x2[pi_i[0] - 1] * (self.fm[str(pi_i[:1])])
            for i in range(1, n):
                latt_pti = np.sort(pi_i[:i + 1])
                latt_ptimin1 = np.sort(pi_i[:i])
                ch = ch + x2[pi_i[i] - 1] * (self.fm[str(latt_pti)] - self.fm[str(latt_ptimin1)])
            return ch
        else:
            print("If using sugeno measure, you need to use chi_sugeno.")


    def produce_lattice(self):
        """
            This method builds is where the lattice(or FM variables) will be learned.
            The FM values can be found via a quadratic program, which is used here
            after setting up constraint matrices. Refer to papers for complete overview.
        :return: Lattice, the learned FM variables.
        """

        fm_len = 2 ** self.N - 1  # nc
        E = np.zeros((fm_len, fm_len))  # D
        L = np.zeros(fm_len)  # f
        index_keys = self.get_keys_index()
        for i in range(0, self.M):  # it's going through one sample at a time.
            l = self.trainLabels[i]  # this is the labels
            fm_coeff = self.get_fm_class_img_coeff(index_keys, self.trainSamples[:, i], fm_len)  # this is Hdiff
            # print(fm_coeff)
            L = L + (-2) * l * fm_coeff
            E = E + np.matmul(fm_coeff.reshape((fm_len, 1)), fm_coeff.reshape((1, fm_len)))

        G, h, A, b = self.build_constraint_matrices(index_keys, fm_len)
        solvers_qp = silent(solvers.qp)
        sol = solvers_qp(matrix(2 * E, tc='d'), matrix(L.T, tc='d'), matrix(G, tc='d'), matrix(h, tc='d'),
                         matrix(A, tc='d'), matrix(b, tc='d'))
        g = sol['x']
        Lattice = {}
        for key in index_keys.keys():
            Lattice[key] = g[index_keys[key]]
        return Lattice


    def build_constraint_matrices(self, index_keys, fm_len):
        """
        This method builds the necessary constraint matrices.
        :param index_keys: map to reference lattice components
        :param fm_len: length of the fuzzy measure
        :return: the constraint matrices
        """

        vls = np.arange(1, self.N + 1)
        line = np.zeros(fm_len)
        G = line
        line[index_keys[str(np.array([1]))]] = -1.
        h = np.array([0])
        for i in range(2, self.N + 1):
            line = np.zeros(fm_len)
            line[index_keys[str(np.array([i]))]] = -1.
            G = np.vstack((G, line))
            h = np.vstack((h, np.array([0])))
        for i in range(2, self.N + 1):
            parent = np.array(list(itertools.combinations(vls, i)))
            for latt_pt in parent:
                for j in range(len(latt_pt) - 1, len(latt_pt)):
                    children = np.array(list(itertools.combinations(latt_pt, j)))
                    for latt_ch in children:
                        line = np.zeros(fm_len)
                        line[index_keys[str(latt_ch)]] = 1.
                        line[index_keys[str(latt_pt)]] = -1.
                        G = np.vstack((G, line))
                        h = np.vstack((h, np.array([0])))

        line = np.zeros(fm_len)
        line[index_keys[str(vls)]] = 1.
        G = np.vstack((G, line))
        h = np.vstack((h, np.array([1])))

        # equality constraints
        A = np.zeros((1, fm_len))
        A[0, -1] = 1
        b = np.array([1]);

        return G, h, A, b


    def get_fm_class_img_coeff(self, Lattice, h, fm_len):  # Lattice is FM_name_and_index, h is the samples, fm_len
        """
        This creates a FM map with the name as the key and the index as the value
        :param Lattice: dictionary with FM
        :param h: sample
        :param fm_len: fm length
        :return: the fm_coeff
        """

        n = len(h)  # len(h) is the number of the samples
        fm_coeff = np.zeros(fm_len)
        pi_i = np.argsort(-h) + 1
        for i in range(1, n):
            fm_coeff[Lattice[str(np.sort(pi_i[:i]))]] = h[pi_i[i - 1] - 1] - h[pi_i[i] - 1]
        fm_coeff[Lattice[str(np.sort(pi_i[:n]))]] = h[pi_i[n - 1] - 1]
        np.matmul(fm_coeff, np.transpose(fm_coeff))
        return fm_coeff


    def get_keys_index(self):
        """
        Sets up a dictionary for referencing FM.
        :return: The keys to the dictionary
        """

        vls = np.arange(1, self.N + 1)
        count = 0
        Lattice = {}
        for i in range(0, self.N):
            Lattice[str(np.array([vls[i]]))] = count
            count = count + 1
        for i in range(2, self.N + 1):
            A = np.array(list(itertools.combinations(vls, i)))
            for latt_pt in A:
                Lattice[str(latt_pt)] = count
                count = count + 1
        return Lattice



In [3]:
# Functions

def create_dataset(dim, all_perms, num_per_perm, superset_factor):
    """
    Create a dataset with all possible permutation, with each permutation having the same number of samples.
    
    :param dim: Dimension of data sample
    :param all_perms: A list of permutation. 
                       Use as an input so that the algorithm creates data for different permutations in the order assigned.
    :param num_per_perm: Number of data samples for each permutation
    :param superset_factor: Data for each permutation are pulled from a randomly generated dataset. 
                            To ensure that each permutation gets at least #num_per_perm# data samples,
                            create a dataset #superset_factor* times (normally 3 will be enough) bigger than the dataset wanted.
    """
    # Every permutation gets the same number of train/test data samples,
    # To ensure that, calculate the number of data needed in total, 
    # and generate a super dataset that is multiple times bigger.
    num = math.factorial(dim) * num_per_perm
    # Create superset
    data_superset = np.random.rand(dim, num*superset_factor)
    # Get permutation of each data sample
    data_perms = np.argsort(data_superset, 0)
    # N! possible permutations
#     all_perms = list(permutations(list(range(dim))))
    # Group data sample according to its permutation
    data_idx_superset_by_perm = []
    
    for i, current_perm in enumerate(all_perms):
        # Get index of data sample of certain permutation and save to list
        temp = np.where(data_perms[0, :]==current_perm[0])
        for idx, p in enumerate(current_perm):
            temp = np.intersect1d(temp, np.where(data_perms[idx, :]==p))
        if temp.size < num_per_perm:
            print('Current permutation doesn\'t have sufficient number of samples. Please regenerate!')
            exit()
        data_idx_superset_by_perm.append(temp)
    
    # Every permutation gets the same number of train/test data samples,
    # Data is randomly pull from superset each epoch
    data_idx_by_perm = []
    for i in range(len(all_perms)):
        temp = data_idx_superset_by_perm[i]
        random.shuffle(temp)
        data_idx_by_perm.append(temp[0:num_per_perm])
        
    return data_superset, data_idx_superset_by_perm, data_idx_by_perm


def gmean(dim):
    return lambda x, d: np.power(np.prod(x, d), 1/dim)


def w_avg(dim, idx):
    if dim == 3:
        if idx == 0:
            return lambda x, d: np.average(x, d, [0.8, 0.1, 0.1])
        elif idx == 1:
            return lambda x, d: np.average(x, d, [0.2, 0.7, 0.1])
        elif idx == 2:
            return lambda x, d: np.average(x, d, [0.5, 0.2, 0.3])
        elif idx == 3:
            return lambda x, d: np.average(x, d, [0.1, 0.3, 0.5])
        
    elif dim == 4:
        if idx == 0:
            return lambda x, d: np.average(x, d, [0.1, 0.2, 0.3, 0.4])
        elif idx == 1:
            return lambda x, d: np.average(x, d, [0.2, 0.2, 0.2, 0.4])
        elif idx == 2:
            return lambda x, d: np.average(x, d, [0, 0.1, 0.3, 0.6])
        elif idx == 3:
            return lambda x, d: np.average(x, d, [0, 0, 0.4, 0.6])
        


def cal_chi(fm, x):
    """
    Calculates ChI with given fuzzy measure and input
    
    :param fm: Fuzzy measure
    :param x: Input
    :return: Single value Chi output
    """
    pi_i = np.argsort(-x) + 1 # Arg sort of input, with the smallest index being 1
    ch = x[pi_i[0] - 1] * (fm[str(pi_i[:1])])
    for i in range(1, len(x)):
        latt_pti = np.sort(pi_i[:i+1])
        latt_ptimin1 = np.sort(pi_i[:i])
        ch = ch + x[pi_i[i] - 1] * (fm[str(latt_pti)] - fm[str(latt_ptimin1)])
    return ch


def get_cal_chi(fm):
    return lambda x: cal_chi(fm, x)


# Convert decimal to binary string
def sources_and_subsets_nodes(N):
    str1 = "{0:{fill}"+str(N)+"b}"
    a = []
    for i in range(1,2**N):
        a.append(str1.format(i, fill='0'))

    sourcesInNode = []
    sourcesNotInNode = []
    subset = []
    sourceList = list(range(N))
    # find subset nodes of a node
    def node_subset(node, sourcesInNodes):
        return [node - 2**(i) for i in sourcesInNodes]
    
    # convert binary encoded string to integer list
    def string_to_integer_array(s, ch):
        N = len(s) 
        return [(N - i - 1) for i, ltr in enumerate(s) if ltr == ch]
    
    for j in range(len(a)):
        # index from right to left
        idxLR = string_to_integer_array(a[j],'1')
        sourcesInNode.append(idxLR)  
        sourcesNotInNode.append(list(set(sourceList) - set(idxLR)))
        subset.append(node_subset(j,idxLR))

    return sourcesInNode, subset


def get_keys_index(dim):
    """
    Sets up a dictionary for referencing FM.
    :return: The keys to the dictionary
    """
    vls = np.arange(1, dim + 1)
    Lattice = {}
    for i in range(1, dim + 1):
        A = np.array(list(itertools.combinations(vls, i)))
        for latt_pt in A:
            Lattice[str(latt_pt)] = 1
    return Lattice


def get_min_fm_target(dim):
    fm = get_keys_index(dim)
    for key in fm.keys():
        if len(key.split()) != dim:
            fm[key] = 0
        else:
            fm[key] = 1
    return fm
    
    
def get_max_fm_target(dim):
    fm = get_keys_index(dim)
    return fm


def get_mean_fm_target(dim):
    fm = get_keys_index(dim)
    for key in fm.keys():
        fm[key] = len(key.split()) / dim
    return fm


def get_gmean_fm_target(dim):
    fm = get_mean_fm_target(dim)
    return fm


def w_avg_target(dim, weight):
    fm = get_keys_index(dim)
    for idx, key in enumerate(fm.keys()):
        if len(key.split()) == 1:
            fm[key] = weight[idx]
        elif len(key.split()) == 2:
            key1 = int(key[1:-1].split()[0])
            key2 = int(key[1:-1].split()[1])
            fm[key] = weight[key1-1] + weight[key2-1]
        elif len(key.split()) == 3:
            key1 = key[1:-1].split()[0]
            key2 = key[1:-1].split()[1]
            key3 = int(key[1:-1].split()[2])
            key12 = '[' + key1 + ' ' + key2 + ']'
            fm[key] = fm[key12] + weight[key3-1]
    return fm

def get_w_avg_target(weight):
    return lambda dim: w_avg_target(dim, weight)

In [4]:
# Run

# Parameters
num_repetition = 1
dim_list = list(range(3, 5)) 
num_per_perm_train = 100 # Every permutation gets the same number of train/test data samples
num_per_perm_test = 20 # Every permutation gets the same number of train/test data samples
superset_factor = 4 # Create a superset # times larger than what is needed to ensure each class would have sufficient number of samples
eva_funcs = [np.amin, np.amax, np.mean, w_avg(2, 0), w_avg(2, 1), w_avg(2, 2), w_avg(2, 3)] # List of evaluation functions, initialize with dim=2

MSEs_seen_by_dim = []
MSEs_unseen_by_dim = []

FM_by_dim = []

output_dir = 'output/'

# Train&Test for dim = 3 to 8
for dim in dim_list:
#     eva_funcs[-1] = gmean(dim) # List of evaluation functions
    for avg_idx in range(4):
        eva_funcs[avg_idx+3] = w_avg(dim, avg_idx)
    
    all_perms = list(permutations(list(range(dim)))) # N! possible permutations

    # When the # of possible permutations exceed certain number (in here 5!), 
    #instead of feeding only one more permutation a time, feed more.
    train_group_num_limit = math.factorial(5)
    if len(all_perms) > train_group_num_limit:
        step = int(len(all_perms) / train_group_num_limit)
    else:
        step = 1
    
    # Mean Squared Error for each evaluation function, for each percentage, for each repetition, of all test samples, for both seen and unseen data.
    MSEs_seen = np.zeros((len(eva_funcs), len(range(step-1, len(all_perms), step)), num_repetition))
    MSEs_unseen = np.zeros((len(eva_funcs), len(range(step-1, len(all_perms), step))-1, num_repetition))
    # Record FM after train session with both seen and unseen data pattern
    FM = np.zeros((len(eva_funcs), len(range(step-1, len(all_perms), step)), num_repetition, 2**dim-1))
    
    for rep in range(num_repetition):
        print('Repetition:', rep+1)
        random.shuffle(all_perms)
        train_data_superset, _, train_idx_by_perm = create_dataset(dim, all_perms, num_per_perm_train, superset_factor)
        test_data_superset, _, test_idx_by_perm = create_dataset(dim, all_perms, num_per_perm_test, superset_factor)
        
        for perc_idx, perc in enumerate(tqdm(range(step-1, len(all_perms), step))):
            # Find index of train/test sample in superset and shuffle
            train_idx = np.concatenate(train_idx_by_perm[0:perc+1])
            np.random.shuffle(train_idx)
            test_idx = np.concatenate(test_idx_by_perm[0:perc+1])
            # Find data sample through index
            train_d = train_data_superset[:, train_idx]
            test_d = test_data_superset[:, test_idx]
            # Define unseen test data samples when the train data doesn't cover 100% of the permutation
            if perc < len(all_perms)-1:
                test_idx_unseen = np.concatenate(test_idx_by_perm[perc+1:])
                test_d_unseen = test_data_superset[:, test_idx_unseen]
            else:
                test_d_unseen = []
                
            # Define subsets of 'X', or keys for fuzzy measure. Like '1 2' or '1 3 4 5' for g(x1, x2) or g(x1, x3, x4, x5)
            sourcesInNode, subset = sources_and_subsets_nodes(dim)
            keys = [str(np.sort(i)+1) for i in sourcesInNode]
            
            for eva_idx, eva_func in enumerate(eva_funcs):
                # Calculate label with given evaluation function
                train_label = eva_func(train_d, 0)
                test_label = eva_func(test_d, 0)
                # Initialize ChIQP
                chiqp = Choquet_Integral_QP()
                # Train 
                chiqp.train_chi(train_d, train_label)
                # Get fuzzy measure learned
                fm = chiqp.fm
                FM[eva_idx, perc_idx, rep, :] = np.asarray(list(fm.values()))
                # Calculate result from integral with test data
                test_output = np.apply_along_axis(get_cal_chi(fm), 0, test_d)
                MSE = ((test_output - test_label)**2).mean()
                MSEs_seen[eva_idx, perc_idx, rep] = MSE
                # Calculate result from integral with test data - unseen
                if perc < len(all_perms)-1:
                    test_label_unseen = eva_func(test_d_unseen, 0)
                    test_out_unseen = np.apply_along_axis(get_cal_chi(fm), 0, test_d_unseen)
                    MSEs_unseen[eva_idx, perc_idx, rep] = ((test_out_unseen - test_label_unseen)**2).mean()
    FM_by_dim.append(FM)
    MSEs_seen_by_dim.append(MSEs_seen)
    MSEs_unseen_by_dim.append(MSEs_unseen)

with open(output_dir + 'ChIQP_saved_file', 'wb') as f:
    pickle.dump(FM_by_dim, f)
    pickle.dump(MSEs_seen_by_dim, f)
    pickle.dump(MSEs_unseen_by_dim, f)

Repetition: 1


100%|██████████| 6/6 [00:02<00:00,  2.01it/s]


Repetition: 1


  0%|          | 0/24 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [16]:

# Plot for seen data
output_dir = 'output/'
dim_list = list(range(4, 5)) 
eva_funcs = [np.amin, np.amax, np.mean, gmean(2)]

with open(output_dir + 'ChIQP_saved_file', 'rb') as f:
    FM_by_dim = pickle.load(f)
    MSEs_seen_by_dim = pickle.load(f)
    MSEs_unseen_by_dim = pickle.load(f)

for dim, MSEs_seen in zip(dim_list, MSEs_seen_by_dim):
    plot_mean = np.mean(MSEs_seen, 2)
    plot_max = np.max(MSEs_seen, 2)
    plot_min = np.min(MSEs_seen, 2)

    num_perm = math.factorial(dim)

    fig, ax = plt.subplots()

    x = (np.asarray(list(range(1, np.size(MSEs_seen, 1)+1)))) / np.size(MSEs_seen, 1)
    plt.plot(x, plot_mean.transpose())
    ax.set_title('MSE (Data with seen pattern)')
    
    if dim == 3:
        ax.legend(['Min', 'Max', 'Mean', '8 1 1', '2 7 1', '5 2 3', '1 3 5'])
    elif dim == 4:
        ax.legend(['Min', 'Max', 'Mean', '1 2 3 4', '2 2 2 4', '0 1 3 6', '0 0 4 6'])
        
    ax.set_xlabel('Percentage of Seen Data')
    ax.set_ylabel('MSEs avg')
    ax.xaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

    for i in range(len(eva_funcs)):
        plt.fill_between(x, plot_min[i, :], plot_max[i, :], alpha=0.1)
    
    plt.savefig(output_dir + 'ChIQP-' + str(dim) + '-MSE seen.png')
    
    
    
# Plot for unseen data

for dim, MSEs_unseen in zip(dim_list, MSEs_unseen_by_dim):
    plot_mean = np.mean(MSEs_unseen, 2)
    plot_max = np.max(MSEs_unseen, 2)
    plot_min = np.min(MSEs_unseen, 2)

    num_perm = math.factorial(dim)

    fig, ax = plt.subplots()

    x = (np.asarray(list(range(1, np.size(MSEs_unseen, 1)+1)))) / (np.size(MSEs_unseen, 1)+1)
    plt.plot(x, plot_mean.transpose())
    ax.set_title('MSE (Data with unseen pattern)')
    
    if dim == 3:
        print(dim)
        ax.legend(['Min', 'Max', 'Mean', '8 1 1', '2 7 1', '5 2 3', '1 3 5'])
    elif dim == 4:
        print(dim)
        ax.legend(['Min', 'Max', 'Mean', '1 2 3 4', '2 2 2 4', '0 1 3 6', '0 0 4 6'])
        
    ax.set_xlabel('Percentage of Seen Data')
    ax.set_ylabel('MSEs avg')
    ax.xaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

    for i in range(len(eva_funcs)):
        plt.fill_between(x, plot_min[i, :], plot_max[i, :], alpha=0.1)
        
    plt.savefig(output_dir + 'ChIQP-' + str(dim) + '-MSE unseen.png')
    
#     plt.show()

4


In [53]:


for fm, dim in zip(FM_by_dim, dim_list):
    if dim == 3:
        eva_name = ['Min', 'Max', 'Mean', '8 1 1', '2 7 1', '5 2 3', '1 3 5']
        fm_targets = [get_min_fm_target, get_max_fm_target, get_mean_fm_target, get_w_avg_target([.8, .1, .1]), get_w_avg_target([.2, .7, .1]), get_w_avg_target([.5, .2, .3]), get_w_avg_target([.1, .3, .5])]
    if dim == 4:
        eva_name = ['Min', 'Max', 'Mean', '1 2 3 4', '2 2 2 4', '0 1 3 6', '0 0 4 6']
        fm_targets = [get_min_fm_target, get_max_fm_target, get_mean_fm_target, get_w_avg_target([.1, .2, .3, .4]), get_w_avg_target([.2, .2, .2, .4]), get_w_avg_target([0, .1, .3, .6]), get_w_avg_target([0, 0, .4, .6])]
    for eva_idx, fm_target in enumerate(fm_targets):
        
        eva_fm = fm[eva_idx, :, :, :]
        eva_fm_mean = np.mean(eva_fm, 1)
        eva_fm_min = np.amin(eva_fm, 1)
        eva_fm_max = np.amax(eva_fm, 1)
        
        # First set up the figure, the axis, and the plot element we want to animate
        fig = plt.figure()
        ax = plt.axes(xlim=(0, np.size(eva_fm, 2)-1), ylim=(-0.1, 1.1))
        line, = ax.plot([], [], lw=2)
        ax.set_xlabel('Fuzzy Measure Value')
        ax.set_ylabel('Fuzzy Measure Count')
        ax.set_title(eva_name[eva_idx]+' Dim='+str(dim))
        
        # initialization function: plot the background of each frame
        def init():
            x = list(range(np.size(eva_fm, 2)))
            y = list(fm_target(dim).values())
            plt.plot(x, y)
            ax.legend(['FM Target'], loc=4)
            

        # animation function.  This is called sequentially
        def animate(i):
            x = np.asarray(list(range(np.size(eva_fm, 2))))
            y = eva_fm_mean[i, :]
            line.set_data(x, y)
            ax.legend(['FM Predict (Seen data percentage ' + str("{0:.0%}".format((i+1)/np.size(eva_fm, 0))) + ')', 'FM Target'])
            ax.collections = []
            plt.fill_between(x, eva_fm_min[i, :], eva_fm_max[i, :], color='blue', alpha=0.1)
            return line,

        # call the animator.  blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, animate, frames=np.size(eva_fm, 0), init_func=init(), interval=200, blit=True)

        # save the animation as an mp4.  This requires ffmpeg or mencoder to be
        # installed.  The extra_args ensure that the x264 codec is used, so that
        # the video can be embedded in html5.  You may need to adjust this for
        # your system: for more information, see
        # http://matplotlib.sourceforge.net/api/animation_api.html
        anim.save(output_dir + 'ChIQP-' + str(dim) + '-' + eva_name[eva_idx] + 'FM.mp4', fps=3)
#         plt.show()
        plt.close('all')